# 💬 AgRAG

Este notebook configura um pipeline completo de perguntas e respostas (Q&A) com base em um arquivo PDF, usando RAG (Retrieval-Augmented Generation), FAISS, embeddings com HuggingFace, e LLM Gemma via Ollama.

In [ ]:
# 📌 Instalação dos pacotes necessários
%pip install langchain langchain-ollama langchain-huggingface langchain-community faiss-cpu pypdf sentence-transformers ollama pdfplumber

In [2]:
import pdfplumber

def load_pdf_text(path):
    all_text = ""
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            all_text += page.extract_text() + "\n"
    return all_text

pdf_text = load_pdf_text("data/500perguntasmilho.pdf")
print(pdf_text[:1000])


O produtor pergunta, a Embrapa responde
Empresa Brasileira de Pesquisa Agropecuária
Embrapa Milho e Sorgo
Ministério da Agricultura, Pecuária e Abastecimento
O produtor pergunta, a Embrapa responde
José Carlos Cruz
Paulo César Magalhães
Israel Alexandre Pereira Filho
José Aloísio Alves Moreira
Editores Técnicos
Embrapa Informação Tecnológica
Brasília, DF
2011
Exemplares desta publicação podem ser adquiridos na:
Embrapa Informação Tecnológica
Parque Estação Biológica (PqEB), Av. W3 Norte (final)
CEP 70770-901 Brasília, DF
Fone: (61) 3448-4236
Fax: (61) 3448-2494
vendas@sct.embrapa.br
www.embrapa.br/liv
Embrapa Milho e Sorgo
Rodovia MG 424, km 65
Caixa Postal 151
35701-970 Sete Lagoas, MG
Fone: (31) 3779-1000 / 3779-1250
Fax (31): 3779-1088
sac@cnpms.embrapa.br
www.cnpms.embrapa.br
Produção editorial: Embrapa Informação Tecnológica
Coordenação editorial: Fernando do Amaral Pereira
Lucilene Maria de Andrade
Juliana Meireles Fortaleza
Supervisão editorial: Wesley José da Rocha
Revisão de 

In [14]:
# 📌 Quebra de texto e geração dos embeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)
docs = splitter.create_documents([pdf_text])

embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base")
db = FAISS.from_documents(docs, embedding)

In [15]:
# 📌 Configuração do modelo Gemma via Ollama
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="gemma3:1b")  # Requer que o modelo esteja instalado via `ollama pull gemma3:1b`

In [16]:
# 📌 Criação do chain de perguntas e respostas (RAG)
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

template = """
Você é um assistente agrícola especializado na cultura do milho. Responda a pergunta abaixo em português do Brasil, com base exclusivamente no conteúdo fornecido.

Contexto:
{context}

Pergunta: {question}
Resposta:
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=db.as_retriever(search_kwargs={"k": 10}),
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
)

In [17]:
# 📌 Fazer perguntas ao modelo
pergunta = "Como controlar pulgões no milho?"
resposta = qa_chain.invoke(pergunta)
print("Resposta:", resposta["result"])

Resposta: O problema maior é a dificuldade de reconhecer com antecedência a presença da praga. Como uma broca, essa praga, na fase larval, passa a maior parte do tempo dentro do colmo do milho. Uma vez nesse local, fica muito difícil obter eficiência por meio de medidas como a aplicação de inseticidas em pó.

As medidas de controle podem ser divididas em duas categorias principais:

1.  **Controle de plantas daninhas:** A principal forma de controle é eliminar as plantas daninhas que se multiplicam na área. Isso inclui o controle de lagartas-cartucho, que são uma praga comum no milho safrinha, e outras espécies de plantas que competem por recursos, como a lagarta-do-cartucho.

2.  **Controle biológico:**
    *   **Uso de inimigos naturais:** O controle biológico consiste em utilizar insetos benéficos que atacam as pragas. Exemplos incluem o *Sphenalos* (que comem as lagartas), o *Cotesia* (que comem a lagarta-do-cartucho) e o *Lilingula* (que comem a lagarta-do-cartucho).
    *   **Imu